# Exporting a geotiff file from an analysis workflow

**Notebook currently compatible with the `NCI`|`DEA Sandbox` environment only**

### Background

Often, at the end of an analysis, we want to export a geotiff **(outputname.tif)** file, either to save our results, or to allow for exploring our results in a GIS software platform (ArcGIS or QGIS, typically)

### Description

This notebook shows a number of ways to export a geotiff file:
- Exporting a single-band, single time-slice geotiff from an xarray object loaded through a `dc.load` query
- Exporting a multi-band, single time-slice geotiff from an xarray object loaded through a `dc.load` query
- Exporting multiple geotiffs, one for each time-slice of an xarray object loaded through a `dc.load` query
- Exporting a single-band geotiff when handling a simple array and/or an xarray that doesn't contain `transform` or `afffine` metadata


### Technical details
* **Products used:** `Sentinel 2`
* **Analyses used:** `datacube.helpers.write_geotiff` `SpatialTools.array_to_geotiff` `SpatialTools.dataset_to_geotiff`

## Getting started
To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load packages


In [5]:
%matplotlib inline

import datacube
from datacube.helpers import write_geotiff
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import xarray as xr

sys.path.append("../Scripts")
from dea_datahandling import load_ard
import dea_spatialTools

### Connect to the datacube
Give your datacube app a unique name that is consistent with the purpose of the notebook.

In [2]:
dc = datacube.Datacube(app="DEA_writing_to_geotiff")

## Load Sentinel 2 data from the datacube

Here we are loading in a sequence of `Sentinel 2` satellite images through the datacube api. This will provide us with some data to work with.

In [4]:
#create a query
query = {
    'x': (153.40, 153.45),
    'y': (-28.85, -28.90),
    'time': ('2018-01', '2018-06'),
    'measurements': ['nbart_red', 'nbart_green','nbart_blue'],
    'output_crs': 'EPSG:3577',
    'resolution': (-30, 30),
    'group_by': 'solar_day'
}

# Load available data from both Sentinel 2 satellites
ds = load_ard(dc=dc_sentinel2,
              products=['s2a_ard_granule', 's2b_ard_granule'],
              **query)

# Print output data
print(ds)

### Export a single-band, single time-slice geotiff


In [5]:
ds.isel(time = 1).red.write_geotiff

### Export a multi-band, single time-slice geotiff

In [6]:
SpatialTools.dataset_to_geotiff(ds.isel(time=1))

### Export multiple geotiffs, one for each time-slice of an xarray

In [ ]:
for i in range(len(ds.time)):
    date_s2 = str(s2.isel(time=i).time.data)[:-19]
    filename2='{}s2_{}.tif'.format(filename,date_s2)
    print(date_s2)
    SpatialTools.dataset_to_geotiff(filename2, s2.isel(time=i))

### Export a single-band geotiff when handling a simple array

Here we will perform some trivial analysis that will require shifting our datacube acquired xarray object into a numpy array, then we will export our numpy array to a geotiff. This approach is sueful as often in a workflow we don't always have access to the metadata required to work with fuctions like `datacube.helpers.write_geotiff` and this approach will work on any array as long as we know the projection and the coordinates.

In [6]:
#perform some trivial analysis
test_array = np.where(ds.red.values > 1, np.nan, 0)

In [ ]:
#first grab the spatial information from our datacube xarray object
transform, projection = SpatialTools.geotransform(ds, (ds.x, ds.y), epsg=3577)

#export the array
SpatialTools.array_to_geotiff("output.tif",
      test_array, geo_transform = transform, 
      projection = projection, 
      nodata_val=-999)

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/GeoscienceAustralia/dea-notebooks).

**Last modified:** September 2019

**Compatible `datacube` version:** 

In [7]:
print(datacube.__version__)

1.7+43.gc873f3ea


## Tags
Browse all available tags on the DEA User Guide's [Tags Index](https://docs.dea.ga.gov.au/genindex.html)